In [28]:
import numpy as np
import MDAnalysis as mda
import nmrformd

In [29]:
import sys, os, git
current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")

In [30]:
def save_result(data, name = "intra_H2O"):
    """Save the correlation functions in dictionary"""
    if not os.path.exists("raw_data/"):
        os.makedirs("raw_data/")
    saving_file = "raw_data/" + name + ".npy"
    t = data.t
    f = data.f
    C = data.gij[0]
    R1 = data.R1
    R2 = data.R2
    N = data.group_j.atoms.n_atoms
    try:
        previous_dictionary = np.load(saving_file, allow_pickle=True)
        t_prev = np.real(previous_dictionary.item()["t"])
        assert len(t_prev) == len(t)
        C_prev = np.real(previous_dictionary.item()["C"])
        R1_prev = np.real(previous_dictionary.item()["R1"])
        R2_prev = np.real(previous_dictionary.item()["R2"])
        N_prev = np.real(previous_dictionary.item()["N"])
        C = (C*N + C_prev*N_prev) / (N_prev + N)
        R1 = (R1*N + R1_prev*N_prev) / (N_prev + N)
        R2 = (R2*N + R2_prev*N_prev) / (N_prev + N)
        N += N_prev
    except:
        pass
    dictionary = {
    "t": t,
    "f": f,
    "C": C,
    "N": N,
    "R1": R1,
    "R2": R2,
    }
    np.save(saving_file, dictionary)
    return N

In [31]:
while 1<2:
    mystep = 1
    for N in ["N25", "N39", "N62", "N99", "N158", "N251", "N398", "N631", "N1002", "N1589", "N2521"]:
        datapath = git_path + "/nmrformd-data/bulk-water/raw-data/"+N+"/"
        for n in np.arange(1, 5):
            if (os.path.exists(datapath+"prod"+str(n)+".xtc")) & (os.path.exists(datapath+"topology.data")):
                u = mda.Universe(datapath+"topology.data", datapath+"prod"+str(n)+".xtc")
                u.transfer_to_memory(step = mystep)
                hydrogen = u.select_atoms("type 2")
            elif (os.path.exists(datapath+"prod"+str(n)+".xtc")) & (os.path.exists(datapath+"prod1.tpr")):
                u = mda.Universe(datapath+"prod1.tpr", datapath+"prod"+str(n)+".xtc")
                u.transfer_to_memory(step = mystep)
                hydrogen = u.select_atoms("name HW1 HW2")
            n_hydrogen = hydrogen.n_atoms
            intra = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "intra_molecular", number_i=5)
            inter = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "inter_molecular", number_i=1)
            n_intra = save_result(intra, name = N + "_intra_step" + str(mystep))
            n_inter = save_result(inter, name = N + "_inter_step" + str(mystep))
            print(n_hydrogen, mystep, n_intra)
    N = "N4000"
    for mystep in [1, 2, 4, 8, 16, 32]:
        datapath = git_path + "/nmrformd-data/bulk-water/raw-data/"+N+"/"
        u = mda.Universe(datapath+"prod1.tpr", [datapath+"prod1.xtc", datapath+"prod2.xtc"])
        u.transfer_to_memory(step = mystep)
        hydrogen = u.select_atoms("name HW1 HW2")
        n_hydrogen = hydrogen.n_atoms
        intra = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "intra_molecular", number_i=5)
        inter = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "inter_molecular", number_i=1)
        n_intra = save_result(intra, name = N + "_intra_step" + str(mystep))
        n_inter = save_result(inter, name = N + "_inter_step" + str(mystep))
        print(n_hydrogen, mystep, n_intra)

50 1 5 240
50 1 6 288
50 1 7 336
50 1 8 384
78 1 5 380
78 1 6 456
78 1 7 532
78 1 8 608
124 1 5 610
124 1 6 732
124 1 7 854
124 1 8 976
198 1 5 980
198 1 6 1176
198 1 7 1372
198 1 8 1568
316 1 5 1570
316 1 6 1884
316 1 7 2198
316 1 8 2512
502 1 5 2500
502 1 6 3000
502 1 7 3500
502 1 8 4000
796 1 2 1588
796 1 3 2382
796 1 4 3176
796 1 5 3970
1262 1 1 1260
1262 1 2 2520
1262 1 3 3780
1262 1 4 5040
2004 1 1 2002
2004 1 2 4004
2004 1 3 6006
2004 1 4 8008
3178 1 1 3176
3178 1 2 6352
3178 1 3 9528
3178 1 4 12704
5042 1 1 5040
5042 1 2 10080
5042 1 3 15120
5042 1 4 20160
8000 1 1 7998
8000 2 1 7998
8000 4 1 7998
8000 8 1 7998
8000 16 1 7998
8000 32 1 7998
